In [1]:
library(Seurat)
library(ggplot2)
library(dplyr)
library(stringr)
library(tidyr)

Loading required package: SeuratObject

Loading required package: sp

'SeuratObject' was built under R 4.4.1 but the current version is
4.4.2; it is recomended that you reinstall 'SeuratObject' as the ABI
for R may have changed


Attaching package: 'SeuratObject'


The following objects are masked from 'package:base':

    intersect, t



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
data <- readRDS("pbmc_ra_QC.rds")

In [3]:
# Menghitung jumlah total sel per donor_id dan disease
total_cell_count <- data@meta.data %>%
  group_by(donor_id, disease) %>%
  summarise(total_cells = sum(n()), .groups = "drop")  # Total sel dalam setiap sample

In [4]:
# Menggabungkan total_cell_count dengan final_data untuk menghitung proporsi
final_data_with_proportion <- data@meta.data %>%
  group_by(donor_id, disease, rough_annot, fine_annot) %>%
  summarise(cell_count = n(), .groups = "drop") %>%
  left_join(total_cell_count, by = c("donor_id", "disease")) %>%
  mutate(proportion = cell_count / total_cells)  # Hitung proporsi

In [15]:
calculate_p_value <- function(data) {
  # Periksa jika data memiliki dua grup disease
  if(length(unique(data$disease)) == 2) {
    # Tambahkan exact = FALSE untuk menangani ties
    wilcox_test_result <- wilcox.test(cell_count ~ disease, data = data, exact = FALSE)
    return(wilcox_test_result$p.value)
  } else {
    return(NA)  # Jika hanya ada satu grup disease, return NA
  }
}

In [16]:
library(dplyr)
library(purrr)

final_data_with_pvalue <- final_data_with_proportion %>%
  group_split(fine_annot) %>%
  map_dfr(~ data.frame(
    fine_annot = unique(.x$fine_annot),
    p_value = calculate_p_value(.x)
  ))

In [17]:
final_data_with_proportion %>%
  group_by(fine_annot, disease) %>%
  summarise(n_unique = n_distinct(cell_count), .groups = "drop")

fine_annot,disease,n_unique
<fct>,<fct>,<int>
CD4 T central memory,rheumatoid arthritis,17
CD4 T central memory,normal,18
CD4 T effector memory,rheumatoid arthritis,16
CD4 T effector memory,normal,16
CD4 T IFIT,rheumatoid arthritis,6
CD4 T IFIT,normal,6
CD4 T Naive,rheumatoid arthritis,16
CD4 T Naive,normal,16
yd T cells,rheumatoid arthritis,9


In [20]:
# Daftar semua unique fine_annot
fine_annot_list <- unique(final_data_with_proportion$fine_annot)

# Loop untuk setiap fine_annot
for (annot in fine_annot_list) {
  # Filter data untuk fine_annot yang sedang diproses
  plot_data <- final_data_with_proportion %>% filter(fine_annot == annot)
  
  # Membuat plot untuk setiap fine_annot
  p <- ggplot(plot_data, aes(x = disease, y = proportion, fill = disease)) +
    geom_boxplot(outlier.shape = NA) +  # Menyembunyikan outlier di boxplot
    geom_jitter(width = 0.3, size = 7, aes(color = disease)) +  # Menambahkan titik sampel dengan jitter
    theme_minimal() +
    labs(
      title = annot,  # Mengganti judul dengan nama fine_annot
      x = "Disease", 
      y = "Proportion of Cells") +
    theme(
      axis.line = element_line(color = "black"),
      axis.text.x = element_text(size=20),  # Menyembunyikan label sumbu X
      axis.title.x = element_blank(),  # Menyembunyikan judul sumbu X
      axis.text.y = element_text(size=18),  # Menyembunyikan judul sumbu X
      axis.title.y = element_text(size=20),  # Menyembunyikan judul sumbu X
      strip.text = element_text(size = 12, margin = margin(b = 20)),  # Menambah jarak bawah antar facet titles
      plot.title = element_text(size=20, hjust = 0.5),
      legend.position = "none",
      panel.grid = element_blank()
    ) +
    scale_fill_manual(values = c("white", "white")) +  # Setel warna boxplot berdasarkan penyakit
    scale_color_manual(values = c("blue", "orange"))  # Setel warna titik berdasarkan penyakit
  
  # Menambahkan p-value untuk fine_annot yang sedang diproses
  p <- p + geom_text(aes(x = 1.5, y = Inf, label = paste("p-value: ", round(p_value, 3))), 
                     data = final_data_with_pvalue %>% filter(fine_annot == annot), 
                     inherit.aes = FALSE,
                     size = 7, hjust = 0.5, vjust = 3)  # Mengatur posisi p-value

  # Menyimpan grafik untuk setiap fine_annot
  ggsave(paste0(annot, "_proportion.png"), plot = p)
}

Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image


In [54]:
# Daftar kategori dalam rough_annot
rough_annot_levels <- unique(final_data_with_proportion$rough_annot)

# Warna yang sesuai untuk setiap kategori rough_annot
color_palette <- list(
  "NKcells" = c("#730255", "#444DF2"), 
  "CD8Tcells" = c("#053959", "#07B0F2", "#05DBF2"),
  "Bcells" = c("#728C14", "#B5BF6B", "#D97171"),
  "CD4Tcells" = c("#F20544", "#0F88F2", "#F2E74B", "#F2A007", "#F25C05"),
  "Monocytes" = c("#F24162", "#F2ACEE", "#7F58A6", "#F2C12E", "#F2AD94")
)

for (cell_type in rough_annot_levels) {
  
  # Filter data berdasarkan kategori cell_type
  Cell <- final_data_with_proportion %>% 
    filter(rough_annot == cell_type)
  
  # Membuat ggplot untuk setiap sel dengan warna yang sesuai
  p <- ggplot(Cell, 
              aes(x = disease, y = proportion, fill = fine_annot)) +
    # Membuat stacked bar chart
    geom_bar(stat = "identity", position = "fill", width = 0.7) +  
    labs(
      title = cell_type,
      y = "Relative Abundance (%)",
      fill = "Cell Type"
    ) +
    scale_y_continuous(labels = scales::percent_format(accuracy = 1)) +  # Format persentase di sumbu Y
    # Menentukan warna sesuai dengan kategori
    scale_fill_manual(values = color_palette[[cell_type]]) +
    theme_minimal() +
    theme(
      axis.title.x = element_blank(),  # Menampilkan judul sumbu X
      axis.title.y = element_text(size = 12, face = "bold"),  # Menampilkan judul sumbu Y
      axis.text = element_text(size = 12),  # Menyesuaikan ukuran teks sumbu
      legend.title = element_text(size = 16),
      legend.text = element_text(size = 12),
      plot.title = element_text(hjust = 0.5, size = 16, face = "bold"),
      # Menghilangkan gridlines
      panel.grid = element_blank(),  
      # Menambahkan garis sumbu X dan Y
      axis.line = element_line(color = "black", size = 0.3)
    )
  
  # Menyimpan plot ke file PNG dengan nama sesuai dengan rough_annot cell type
  ggsave(paste0(cell_type, "_proportion.png"), plot = p)
}


Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
Saving 7 x 7 in image
